# Big Data Project

## Main

In [ ]:
import DataPrep
import OLS_Regression
import Ridge_Regression
import SVR
import NeuralNetworks
import RandomForest
import KNN

print(DataPrep.report)
import DataPrep

import JupiterPycharmProjekt.OLS_Regression
import JupiterPycharmProjekt.Ridge_Regression
import JupiterPycharmProjekt.SVR
import JupiterPycharmProjekt.NeuralNetworks
import JupiterPycharmProjekt.RandomForest
import JupiterPycharmProjekt.KNN

print(DataPrep.report)

This block executes all model training and evaluation scripts, collects their results and prints the final comparison report.

## Data Preparation  gerne nochmal checken glaube haben dort einiges verändert

### General
In this section the environment for Data Preparation is set up by importing essential Python libraries. Each library plays a key role for the Data Preparation.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import numpy as np

pandas:             Used for handling and analyzing structured data

matplotlib.pyplot:  A fundamental plotting library.

seaborn:            Built on top of matplotlib and simplifies the process of graphical statistics.

sklearn.model_selection + train_test_split: Helps to split a dataset into training and test dataset.

sklearn.preprocessing + LabelEncoder: Transforming data before feeding it into a model.

numpy: Is the foundational package for numerical computing in Python.

### Read original data from CSV

In [ ]:
data = pd.read_csv('../JupiterPycharmProjekt/UsedCarSellingPrices.csv')

This code uses pandas to read the CSV file "Used Car Selling Prices" and loads it into a dataframe called 'data'.

### Label-Encoding for Visualisation before cleaning

#### Define columns that need to be lable-encoded

In [ ]:
categorical_columns = ['fuel', 'seller_type', 'transmission', 'owner']

This line defines a list of column names that represent categorical features in the dataset.

fuel: type of fuel that is used by the car

seller_type: type of car seller

transmission: type of gear

owner: status of ownership

#### Copy data into var label_encoded_data; create empty array lable_encoders

In [ ]:
label_encoded_data = data.copy()
label_encoders = {}

This code sets up the environment for label encoding.

Line1: Copying the dataset

Line2: Initializing the Encoders Dictionary

#### Run lable-encoder for every previosly defined columne (function imported from sklearn)

In [ ]:
from sklearn.preprocessing import LabelEncoder
for col in categorical_columns:
    le = LabelEncoder()
    label_encoded_data[col] = le.fit_transform(label_encoded_data[col])
    label_encoders[col] = dict(zip(le.classes_, le.transform(le.classes_)))

print("\nLabel-Encoded Data for Visualisation:")
print(label_encoded_data.head())

This loop iterates over each categorical column and applies Label Encoding transforming string labels into numeric codes.

1. for col in categorical_columns: Loops through each column listed earlier
2. le = labelEncoder(): Creates a new LabelEncoder instance from scikit-learn for the current column
3. label_encoded_data[col] = le.fit_transform(label_encoded_data[col]): Fits the encoder to the column's categories and transforms them into integers + Replaces the original text values in label_encoded_data with the corresponding numeric labels
4. label_encoders[col] = dict(zip(le.classes_, le.transform(le.classes_))): Stores the mapping of original category names to their encoded values in the label_encoders dictionary + This allows to trace or reverse the encoding later if needed

#### concatinates x and y into one point to be visualized

In [ ]:
all_data_LableEncoded = pd.concat([label_encoded_data], axis=1)

This line creates a new DataFrame called 'all_data_LableEncoded' by concatenating 'label_encoded_data' along the column axis 'axis=1'

#### Drops all columes that are non-numeric to make scaling possible

In [ ]:
all_data_LableEncoded = all_data_LableEncoded.select_dtypes(include=['number'])

This line filters the dataset to keep only the numeric columns from 'all_data_LableEncoded'.

#### Scale data (normalized via MinMaxScaler - between 0 and 1)
sscaler = preprocessing.StandardScaler()    ???

all_data_LableEncoded = sscaler.fit_transform(all_data_LableEncoded)    ???

In [ ]:
nscaler = preprocessing.MinMaxScaler()
all_data_LableEncoded = nscaler.fit_transform(all_data_LableEncoded)

This block performs Min-Max Scaling on the numeric features in the dataset, transforming them into a commonscale between 0 and 1.
1. Line1: Initalizes a MinMaxScaler object from scikit-learn
2. Line2: Calculates the min and max values for each feature iin the dataset and applies the scaling transformation to each value

### Visualisation before cleaning Data

#### Reintegrates Column name for boxplot

In [ ]:
scaled_df = pd.DataFrame(all_data_LableEncoded, columns=label_encoded_data.select_dtypes(include='number').columns)

This line converts the scaled NumPy array (from the Min-Max Scaler) back into a pandas DataFrame and restores the original column names.

#### Boxplot with readable x-axis

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=scaled_df, orient='v', palette='Set2')
plt.xticks(rotation=45)
plt.title("Normed boxplot")
plt.tight_layout()
plt.show()

This block creates a boxplot for each feature in the 'scaled_df' DataFrame to visualize the distribution and spread of the normalized (Min-max scaled) data.
1. Line1: Sets the size of the figure to be 12 inches wide by 6 inches tall and ensures the plot is large enough to accommodate all features without crowding.
2. Line2: Creates a vertical boxplot for each column in the 'scaled_df' DataFrame and uses Seaborn's elegant and color-friendly 'Set2' palette. Each box shows the median, the IQR and Whiskers & Outliers.
3. Line3: Rotates the x-axis labels by 45 degrees for better readability, especially when there are many features.
4. Line4: Adds a title to the plot for context, signaling that the data is normalized.
5. Line5: Adjusts spacing to prevent overlap between axis labels, titles, and plot content.
6. Line6: Renders and displays the final plot in the notebook.

#### Boxplot for only numerical data

In [ ]:
selected_cols = ['selling_price', 'km_driven', 'year']
plt.figure(figsize=(8, 5))
sns.boxplot(data=scaled_df[selected_cols], orient='v', palette='Set3')
plt.title("Normed boxplot for numerical data only")
plt.tight_layout()
plt.show()

This blocks generates a boxplot visualization focused on three specific, scaled numerical features:

'selling_price'

'km_driven'

'year'

1. Line1: Selects the subset of important numerical features for focused analysis.
2. Line2: Sets the plot size to be 8 inches wide and 5 inches tall.
3. Line3: Creates a vertical boxplot for just the selected columns using the soft, pastel 'Set3' color palette from Seaborn.
4. Line4: Adds a descriptive title to clarify that this plot shows normalized (scaled) numerical features.
5. Line5: Ensures layout is adjusted for neatness and then displays the plot.

#### Pairplot to show correlation

In [ ]:
sns.pairplot(scaled_df[selected_cols])
plt.suptitle("Pairplot for select charactaristics", y=1.02)
plt.show()

This blocks creates a pairplot to visually explore pairwise relationships among the selected numerical features:

'selling_price'

'km_driven'

'year'

1. Line1: Creates a grid of scatterplots for each pairwise combination of the selected features. This helps to visualize Correlations, Clustering tendencies and Linearity or Non-Linearity Relationships. The Histograms are shown on the diagonal to represent each variable's distribution.
2. Line2: Adds a super title above the entire plot grid.
3. Line3: Renders the entire pairplot for viewing.

### Clean Data & Create variable Brand

#### Remove missing data

In [ ]:
data = data.dropna()

This line removes all rows with missing values from the 'data' DataFrame.

#### Show how much data was removed

In [ ]:
print("Data after removing data:")
print(data.isnull().sum())
print(f"Remaining rows: {len(data)}")

This block checks and confirms that all missing values have been removed from the dataset and reports the number of remaining rows.

1. Line1: Prints a header to indicate that the following output relates to the cleaned dataset.
2. Line2: Checks for missing values in each column of the 'data' DataFrame, creates a Boolean mask of the same shape as the data and then counts the number of 'True' values in each column, i.e. the number of missing entries.
3. Line3: Prints the total number of rows left in the dataset after dropping rows with missing values using 'len(data)'.

#### IQR-based Removal of Outliers

In [ ]:
def remove_outliers_iqr(df, column):
    Q1 = np.percentile(df[column], 25)
    Q3 = np.percentile(df[column], 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

This function removes outliers from a specific column in a DataFrame using the Interquartile Range (IQR) method

1. 'Q1': 25th percentile - the value below which 25% of the data falls
2. 'Q3': 75th percentile - the value below which 75% of the data falls
3. 'IQR = Q3-Q1': IQR is the spread of the middle 50% of values and it is used to understand the natural range of variation in the data.
4. Line5 + Line6: These define the acceptable range and any values below the lower bound or above the upper bound are considered outliers.
5. Line7: Returns a filtered version of the original DataFrame, keeping only the rows where the specified column's value is within the acceptable range -> Outlier are removed.

#### Apply to most important column

In [ ]:
data = remove_outliers_iqr(data, 'selling_price')
data = remove_outliers_iqr(data, 'km_driven')

These lines apply the IQR-based outlier removal function to two important columns in the dataset: 'selling_price' and 'km_driven'.

1. Line1: Removes rows where 'selling_price' is considered an outlier based on the IQR rule and keeps only cars with selling prices within the typical range.
2. Line2: Applies the same IQR filtering to the 'km_driven' column and eliminates unusually low or high mileage entries that could distort statistical analysis or model training.

In [ ]:
print("\nDaten nach IQR-basierter Ausreißerbereinigung:")
print(f"Max. Verkaufspreis: {data['selling_price'].max()}")
print(f"Max. Kilometerstand: {data['km_driven'].max()}")
print(f"Verbleibende Zeilen nach IQR-Filter: {len(data)}")

This block prints a quick summary of the dataset after removing outliers using the IQR method.
1. Line1: Prints a headline used for clarity when reading the console output.
2. Line2: Displays the maximum selling price in the cleaned dataset which helps verifying that extremely high prices have been removed.
3. Line3: Shows the maximum odometer reading after outlier removal and ensures that unusally high mileage values have been filtered out.
4. Line4: Prints the number of remaining rows in the dataset which tells how much data is left after removing rows that contained outliers in 'selling_price' and 'km_driven'.

#### Create 'Brand' as new column

In [ ]:
data['brand'] = data['name'].str.split().str[0]
print(data)

This line creates a new column called 'brand' by extracting the first word from the 'name' column which represents the car brand.

### Label-Encoding for Visualisation after cleaning

#### ???

In [ ]:
categorical_columns = ['fuel', 'seller_type', 'transmission', 'owner']
label_encoded_data = data.copy()
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    label_encoded_data[col] = le.fit_transform(label_encoded_data[col])
    label_encoders[col] = dict(zip(le.classes_, le.transform(le.classes_)))

print("\nLabel-Encoded Data (nur zur Referenz):")
print(label_encoded_data.head())

This block performs label encoding on selected categorical columns, converting them from text values to integers so they can be used in regression models.

1. Line1: Specifies the list of categorical features to be encoded which are typically textual descriptors that must be converted into numeric format for modeling.
2. Line2: Creates a copy of the original dataset to apply the encodings without altering the raw data.
3. Line3: Initializes an empty dictionary to store the encoding mappings for each categorical column.
4. The 'for' loop: Iterates over each column in the 'categorical_columns' list; The 'LabelEncoder()' from scikit-learn is used to convert category labels into integers. The transformed values replace the original column in 'label_encoded_data'. The mapping of original class labels to integer codes is stored in 'label_encoders' for reference or inverse transformation later.
5. Line10 + Line11: Displays the first few rows of the updated dataset to confirm that the categorical features have been encoded.

#### concatinate x and y into one point to be visualized

In [ ]:
all_data_LableEncoded = pd.concat([label_encoded_data], axis=1)

This line creates a new DataFrame called 'all_data_LableEncoded' by concatenating the contents of 'label_encoded_data' along the columns axis.

#### Drop all columns that are non-numeric to make scaling possible

In [ ]:
all_data_LableEncoded = all_data_LableEncoded.select_dtypes(include=['number'])

This line filters the 'all_data_LableEncoded' DataFrame to include only numeric columns, removing any that are not numeric.

#### Scale data (normalized)

sscaler = preprocessing.StandardScaler()

all_data_LableEncoded = sscaler.fit_transform(all_data_LableEncoded)

In [ ]:
nscaler = preprocessing.MinMaxScaler()
all_data_LableEncoded = nscaler.fit_transform(all_data_LableEncoded)

This code applies Min-Max Scaling to normalize all numeric features in the dataset, transforming their values to a common range between 0 and 1.
1. Line1: Initializes a MinMaxScaler object from scikit-learn's 'preprocessing' module and will scale each feature individually.
2. Line2: Calculates the minimum and maximum values for each feature and scales each value in the dataset to the 0-1 range.

### Visualization after Data cleaning

#### Scale Data ???

In [ ]:
scaled_df = pd.DataFrame(all_data_LableEncoded, columns=label_encoded_data.select_dtypes(include='number').columns)

This line converts the scaled NumPy array back into a pandas DataFrame and restores the original column names, making the data human-readable and easier to work with.

#### 1. Boxplot with readable axis names

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=scaled_df, orient='v', palette='Set2')
plt.xticks(rotation=45)
plt.title("Boxplot der skalierten numerischen Features")
plt.tight_layout()
plt.show()

This block creates a boxplot for all features in the 'scaled_df' DataFrame, which contains only scaled numeric data. The plot helps visually inspecting the distribution and variability of each feature.
1. Line1: Sets the figure size to 12 inches wide by 6 inches tall for better readability.
2. Line2: Uses Seaborn to draw vertical boxplots for each numeric feature in 'scaled_df' and 'palette=Set2' gives the plot a soft, color-coded appearance to distinguish features visually.
3. Line3: Rotates the x-axis labels by 45 degrees so that long feature names dont overlap and remain legible.
4. Line4: Adds a descriptive title
5. Line5 + Line6: Adjusts the layout to avoid overlapping elements and displays the final plot.

#### 2. Boxplot only for selected numeric columns

In [ ]:
selected_cols = ['selling_price', 'km_driven', 'year']
plt.figure(figsize=(8, 5))
sns.boxplot(data=scaled_df[selected_cols], orient='v', palette='Set3')
plt.title("Boxplot ausgewählter Merkmale")
plt.tight_layout()
plt.show()

This block creates a boxplot visualization for a selected subset of key numeric features: 'selling_price', 'km_driven' and 'year', all of which have been previously scaled to 0-1 range.
1. Line1: Selects the three features for targeted visualization.
2. Line2: Sets the figure size to 8 inches wide and 5 inches tall for compact clarity.
3. Line3: Draws vertical boxplots for just the selected columns using Seaborn's pastel 'Set3' color palette.
4. Line4: Adds a title.
5. Line5 + Line6: Adjusts spacing to prevent overlap and displays the plot.

#### 3. Pairplot for Distribution and Correlation

In [ ]:
sns.pairplot(scaled_df[selected_cols])
plt.suptitle("Paarweise Verteilungen ausgewählter Merkmale", y=1.02)
plt.show()
#print("Trainingsdaten zusätzlich als 'prepared_used_car_data_train.parquet' gespeichert.")
#print("Testdaten zusätzlich als 'prepared_used_car_data_test.parquet' gespeichert.")
#print("Gesamtdaten zusätzlich als 'prepared_used_car_data.parquet' gespeichert.")

This block creates a pairplot that visualizes the pairwise relationships and distributions of three selected, scaled features: 'selling_price', 'km_driven' and 'year'.
1. Line1: Generates a grid of plots: Scatter plots and Histograms.
2. Line2: Adds a descriptive title above the plot grid; 'y=1.02' adjusts the title position slightly above the plot area to prevent overlap.
3. Line3: Renders and displays the plot.

### One-Hot-Encoding for Regression Model Training & Testing

#### One-Hot-Encoding for categorical Variables

In [ ]:
encoded_data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

print("\nOne-Hot-Encoded Data:")
print(encoded_data.head())

print(encoded_data)

This block uses one-hot encoding to transform categorical columns in the dataset into binary 0 or 1 columns, making them suitable for regression models.
1. Line1: Performs one-hot encoding on the columns listed in 'categorical_columns'. For each unique category in these columns, new binary columns are created. Drops the first category for each column to avoid multicollinearity when using models like linear regression.
2. Line3: Prints a header for clarity in console output.
3. Line4: Displays the first few rows of the encoded dataset for a quick preview.
4. Line6: Prints the entire DataFrame, which now contains both numeric and one-hot encoded binary columns.

### Sort data by 'year' and 'km_driven'

In [ ]:
encoded_data_sorted = encoded_data.sort_values(by=['year', 'km_driven'], ascending=[False, True])

This line sorts the encoded dataset based on two columns - 'year' and 'km_driven' - to organize the data in a meaningful order.

### Prepare data used for regression analysis

In [ ]:
features = encoded_data_sorted.columns.drop(['name', 'selling_price'])
target = 'selling_price'

X = encoded_data_sorted[features]
y = encoded_data_sorted[target]

This block prepares the feature matrix 'X' and target vector 'Y' for training a regression model, using sorted and one-hot encoded dataset.
1. Line1: Selects all column names except: 'name' and 'selling_price' -> Result is a list of input features for the model.
2. Line2: Explicitly defines 'selling_price' as the target variable.
3. Line4: Creates the feature matrix 'X' by selecting only the columns in 'features' from the dataset; 'X' will be used as input for the regression model
4. Line5: Creates the target vector 'Y', which contains the selling prices (values to be predicted)

#### Saving test data

In [ ]:
all_data = pd.concat([X, y], axis=1)
all_data.to_csv('prepared_used_car_data_all.csv', index=False)

This block recombines the feature 'X' and target 'Y' into a single DataFrame and saves it as a '.csv' file for future use.
1. Line1: Concatenates the feature matrix 'X' and the target vector 'Y' horizontally and reconstructs the full dataset 'all_data' with both inputs and outputs in one table.
2. Line2: Saves the combined dataset to a CSV file and ensures that the DataFrame index is not written to the file, keeping the output clean and suitable for reuse.

#### Creation of training and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

This line uses scikit-learn's 'train_test_split()' function to divide the dataset into training and testing subsets, a crucial step for evaluating regression models.

#### Saving of training data

In [ ]:
train_data = pd.concat([X_train, y_train], axis=1)
train_data.to_csv('prepared_used_car_data_train.csv', index=False)

This block combines the training features and labels into a single DataFrame and then exports it to a '.csv' file for storage or reuse.
1. Line1: Merges the training input features 'X_train' and then training target values 'Y_train' side by side (along columns) and produces a single DataFrame 'train_data' that contains all the necessary data for training a model.
2. Line2: Saves the 'train_data' DataFrame as a CSV file and ensures the row indices are not written into the file, keeping it clean and easy to reload.

#### Saving of test data

In [ ]:
test_data = pd.concat([X_test, y_test], axis=1)
test_data.to_csv('prepared_used_car_data_test.csv', index=False)

This code combines the test features and labels into a single DataFrame and then saves it as a CSV file for future use or evaluation.
1. Line1: Merges the test feature set 'X_test' and the corresponding target values 'Y_test' horizontally and produces a new DataFrame 'test_data' that includes all the columns needed to evaluate a regression model.
2. Line2: Saves the resulting test dataset to a CSV file and prevents the row index from being included in the file, making the CSV clean and readable.

#### Confirming saved files

In [ ]:
print("\nTrainingsdaten gespeichert als 'prepared_used_car_data_train.csv'")
print("Testdaten gespeichert als 'prepared_used_car_data_test.csv'")

These print statements simply confirm to the user that the training and test datasets have been successfully saved to CSV files.
1. Line1: Outputs a message confirming that the training data was saved.
2. Line2: Confirms that the test data was also saved.

## KNN

#### Set Up

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
import DataPrep

(X_train_KNN, X_test_KNN, Y_train_KNN, Y_test_KNN) = train_test_split(DataPrep.X_LE, DataPrep.Y_LE, test_size=0.2, random_state=42)

This block sets upo the environment to train and evaluate a K-Nearest Neighbors Regression model using a train/test split of preprocessed data.

1. Line1: Imports the KNN regressor from scikit-learn
2. Line2: Adds a tool for hyperparameter tuning by trying different values and splits the dataset into training and testing sets
3. Line3: Imports the DataPrep file
4. Line5: Splits features and target into a training set and testing set while ensuring that the split is reproducible

#### Initialize KNN model

In [ ]:
knnmodelCV = KNeighborsRegressor()

This line creates an instance of the KNN model from scikit-learn with default parameters.

#### Define grid of neighbor counts

In [ ]:
param_grid = {
'n_neighbors': range(3, 22, 2),
'weights': ['uniform', 'distance'],
'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
'leaf_size': [8, 16, 32, 64, 128, 256, 512],
'p': [2, 3, 4, 5, 6, 7, 8]

}

This block defines a hyperparameter grid that will be used to tune a KNN regressor using tools like GridSearchCV.

1. Line2: Tries odd values for k from 3 to 21 which defines how many neighbors the model considers when making predictions.
2. Line3: 'uniform' makes sure all neighbors contribute equally to the prediction and 'distance' makes sure that closer neighbors contribute more to the prediction than distant ones.
3. Line4: Specifies the search algorithm used to find neighbors. 'auto' -> Chooses the best algorithm based on the data. 'ball_tree' and 'kd_tree' -> Use tree structure for fast searches. 'brute' -> Calculates distances directly.
4. Line5: Affects the speed vs. memory tradeoff in tree based algorithms. Smaller values result in faster query time. Larger values result in faster tree building.
5. Line6: Specifies the power parameter for the Minkowski distance.

#### Run 10-fold cross-validation across neighbor settings

In [ ]:
CV_knnmodel = GridSearchCV(estimator=knnmodelCV, param_grid=param_grid, cv=10,n_jobs=-1)
CV_knnmodel.fit(X_train_KNN, Y_train_KNN)

This code uses GridSearchCV from scikit-learn to perform an exhaustive search over a defined set of hyperparameters for a KNN regression model.
Assigns all available CPU cores to parallelize the search and speed up computation and uses 10-fold cross-validation.

#### Output the best number of neighbors

In [ ]:
print("Best parameters found:", CV_knnmodel.best_params_)

This line prints the optimal set of hyperparameters that were found during the GridSearchCV process.

#### Evaluating Model Performance

In [ ]:
Y_train_pred = CV_knnmodel.predict(X_train_KNN)
Y_train_dev = sum((Y_train_KNN - Y_train_pred)**2)
Y_train_meandev = sum((Y_train_KNN - Y_train_KNN.mean())**2)
r2 = 1 - Y_train_dev / Y_train_meandev

This block manually calculates the coefficient of determination (r2) to evaluate how well the KNN model fits the training data.
1. Line1: Uses the trained cross-validated KNN model to predict selling prices for the training set.
2. Line2: Calculates the residual sum of squares (RSS) and measures how far off the predictions are from the actual values which should be minimal.
3. Line3: Calculates the total sum of squares (TSS) and measures the total variation in the target variable (how far are values from the mean)
4. Line4: Calculates the coefficient of determination (r2).

#### Predict on test set

In [ ]:
Y_test_pred = CV_knnmodel.predict(X_test_KNN)
Y_test_dev = sum((Y_test_KNN - Y_test_pred)**2)
Y_test_meandev = sum((Y_test_KNN - Y_test_KNN.mean())**2)
pseudor2 = 1 - Y_test_dev / Y_test_meandev

DataPrep.report.loc[len(DataPrep.report)] = ["KNN_LE ", r2, pseudor2,"", CV_knnmodel.cv_results_['mean_test_score'][CV_knnmodel.best_index_], CV_knnmodel.cv_results_['std_test_score'][CV_knnmodel.best_index_]]

This block evaluates the KNN model on the test set using the coefficient of determination and calculates a pseudo r2 score for generalization performance and appends a summary row to report DataFrame for later comparison with other models.
1. Line1: Uses the best tuned KNN model to predict the selling prices for the test set.
2. Line2: Calculates the residual sum of squares (RSS) on the test data.
3. Line3: Computes the total sum of squares (TSS) of the actual test values
4. Line4: Calculates pseudo-r2 which is the models explanatory power on the unseen test data.
5. Line6: Logs results to a report.

#### Splitting One-Hot Encoded Data for Training and Testing

In [ ]:
X_train_KNN, X_test_KNN, Y_train_KNN, Y_test_KNN) = train_test_split(DataPrep.X_OH, DataPrep.Y_OH, test_size=0.2, random_state=42)

This line splits the one-hot encoded dataset into training and testing sets for building and evaluating a KNN model.

#### Run 10-fold cross-validation across neighbor settings

In [ ]:
CV_knnmodel = GridSearchCV(estimator=knnmodelCV, param_grid=param_grid, cv=10,n_jobs=-1)
CV_knnmodel.fit(X_train_KNN, Y_train_KNN)

This block performs hyperparameter optimization for the KNN regression model using one-hot encoded data with cross validation to ensure reliable model selection.
1. Line1: Performs an exhaustive grid search over the parameter combinations defined in 'param_grid'.
2. Line2: Trains and evaluates all hyperparameter combinations using the training data based on one-hot encoded features.

#### Output the best number of neighbors

In [ ]:
print("Best parameters found:", CV_knnmodel.best_params_)
Y_train_pred = CV_knnmodel.predict(X_train_KNN)
Y_train_dev = sum((Y_train_KNN - Y_train_pred)**2)
Y_train_meandev = sum((Y_train_KNN - Y_train_KNN.mean())**2)
r2 = 1 - Y_train_dev / Y_train_meandev

This line prints the optimal hyperparameter combination selected by 'GridSearchCV' for the KNN regression model trained on one-hot encoded data.
1. Line2: Predicts the selling prices for the training data using the best-found KNN model.
2. Line3: Computes the residual sum of squares (RSS) and measures how much error remains after using the models predictions.
3. Line4: Computes the total sum of squares (TSS) and represents the total variance in the training labels serving as a baseline.
4. Line5: Calculates the r2 score which explains how much variation is explained by the model.

#### Predict on test set

In [ ]:
Y_test_pred = CV_knnmodel.predict(X_test_KNN)
Y_test_dev = sum((Y_test_KNN - Y_test_pred)**2)
Y_test_meandev = sum((Y_test_KNN - Y_test_KNN.mean())**2)
pseudor2 = 1 - Y_test_dev / Y_test_meandev

This block evaluates how well the tuned KNN model generalizes to unseen data by manually calculating the pseudo r2 score.
1. Line1: Uses the trained KNN model to predict selling prices for the test set.
2. Line2: Computes the residual sum of squares (RSS) for the test set and measures the models prediction error on unseen data.
3. Line3: Compues the total sum of squares (TSS) for the test set and represents the total variance in the actual test values serving as a baseline.
4. Line4: Calculates the pseudo-r2 and indicates how well the model generalizes to new unseen data.

#### Log KNN Model results in a report

In [ ]:
DataPrep.report.loc[len(DataPrep.report)] = ["KNN_OH ", r2, pseudor2,"", CV_knnmodel.cv_results_['mean_test_score'][CV_knnmodel.best_index_], CV_knnmodel.cv_results_['std_test_score'][CV_knnmodel.best_index_]]
print(DataPrep.report.head())
print(CV_knnmodel.best_params_)

This block logs the performance metrics of the KNN model trained with one-hot encoded data into a centralized evaluation report and then displays the report and the best hyperparameters.

## Neural Networks

#### Preparation to train a Neural Network

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
import DataPrep

This block sets up the environment for training and tuning a neural network regression model using scikit-learn's 'MLPRegressor' with data and configuration coming from a shared module called 'DataPrep'.
1. Line1: Used to split data into training and testing sets and enables hyperparameter tuning using exhaustive search with cross-validation.
2. Line2: Imports 'MLPRegressor' a multilayer perceptron for regression tasks.
3. Line3: Imports the 'DataPrep' module.

#### Splitting Label-Encoded Data

In [ ]:
(X_train_nn, X_test_nn, Y_train_nn, Y_test_nn) = train_test_split(DataPrep.X_LE, DataPrep.Y_LE, test_size=0.2, random_state=42)

This line splits the label-encoded dataset into training and testing sets preparing it for use in training a neural network regression model.

#### Defining a Hyperparameter Grid for Tuning the Network Regressor

In [ ]:
param_grid = {
'hidden_layer_sizes': [(5,), (8,), (10,), (13,)],
'alpha': [0.0, 0.0025, 0.005, 0.0075, 0.01, 0.1],
'activation': ['logistic', 'tanh', 'relu'],
'solver': ['sgd', 'adam', 'lbfgs'],
'max_iter': [5000],
'random_state': [0],
'learning_rate': ['constant', 'invscaling', 'adaptive'],
}

This block defines a parameter grid to be used with 'GridSearchCV' for tuning a 'MLPRegressor' model in a regression task. Each key represents a hyperparameter and each value is a list of options that 'GridSearchCV' will explore during cross-validation.
1. Line2: Defines the structure of the hidden layers; how many neurons
2. Line3: L2 regularization term which helps to prevent overfitting by penalizing large weights
3. Line4: Specifies the activation function used in the hidden layers
4. Line5: Optimization algorithm used for training
5. Line6: Sets the maximum number of training iterations to 5000
6. Line7: Ensures reproducibility by fixing the random seed used inernally by the model
6. Line8: Controls how the learning rate adapts over time when using 'sgd' or 'adam'

#### Training a Neural Network with Grid Search and Cross-Validation

In [ ]:
NNetRregCV = MLPRegressor()
CV_nnmodel = GridSearchCV(estimator=NNetRregCV, param_grid=param_grid, cv=10,n_jobs=-1)
CV_nnmodel.fit(X_train_nn, Y_train_nn)

This block creates, tunes and trains a neural network regressor using scikit-learn's 'MLPRegressor' and 'GridSearchCV'.
1. Line1: Initializes a Multi-Layer Perceptron (MLP) Regressor which is a type of feedforward neural network.
2. Line2: Wraps the neural network model with 'GridSearchCV' to perform automated hyperparameter tuning.
3. Line3: Trains the model using the training data and runs the grid search.

#### Evaluating the Neural Network on Training Data

In [ ]:
Y_train_pred = CV_nnmodel.predict(X_train_nn)
Y_train_dev = sum((Y_train_nn - Y_train_pred)**2)
Y_train_meandev = sum((Y_train_nn - Y_train_nn.mean())**2)  # [aus PDF]
r2 = 1 - Y_train_dev / Y_train_meandev  # [aus PDF]

This block manually calculates the r2 score to evaluate how well the trained neural network model fits the training data.
1. Line1: Uses the best neural network model selected by 'GridSearchCV' to predict target values for the training set
2. Line2: Calculates the residual sum of squares (RSS) and measures the total prediction error of the model
3. Line3: Calculates the total sum of squares (TSS) and measures how much variance is in the target data without any model
4. Line4: Computes the r2 score which explains the variance in the training data

#### Evaluating the Neural Network on Test Data

In [ ]:
Y_test_pred = CV_nnmodel.predict(X_test_nn)
Y_test_dev = sum((Y_test_nn - Y_test_pred)**2)
Y_train_meandev = sum((Y_test_nn - Y_test_nn.mean())**2)  # [aus PDF]
pseudor2 = 1 - Y_test_dev / Y_train_meandev

This block calculates the pseudo r2 score to assess how well the trained neural network regressor performs on unseen test data.
1. Line1: Predicts target values for the test set using the neural network model selected by 'GridSearchCV'
2. Line2: Computes the residual sum of squares (RSS) for the test set which represents the total prediction error of the model on unseen data
3. Line3: Computes the total sum of squares (TSS) for the test set which represents the total variance in the target values
4. Line4: Calculates the pseudo r2 score used to evaluate the model performance on the test data by measuring how much variability in the test target data is explained by the model

#### Logging Neural Network Results into the Model Comparison Report

In [ ]:
DataPrep.report.loc[len(DataPrep.report)] = ["NN_LE ", r2, pseudor2,"", CV_nnmodel.cv_results_['mean_test_score'][CV_nnmodel.best_index_], CV_nnmodel.cv_results_['std_test_score'][CV_nnmodel.best_index_]]

This line adds a new row to the shared 'DataPrep.report' table documenting the performance metrics of the neural network model trained with label-encoded data.

#### Displaying the best hyperparameters

In [ ]:
print(CV_nnmodel.best_params_)

This line prints the optimal hyperparameter configuration found by 'GridSearchCV' during training of the MLPRegressor

#### Splitting One-Hot Encoded Data for Neural Network Training and Testing

In [ ]:
(X_train_nn, X_test_nn, Y_train_nn, Y_test_nn) = train_test_split(DataPrep.X_OH, DataPrep.Y_OH, test_size=0.2, random_state=42)

This line splits the dataset - specifically the one-hot encoded version - into a training set and a test set preparing it for use with a neural network regression model

#### Grid Search to Train and Tune a Neural Network with One-Hot Encoded Data

In [ ]:
CV_nnmodel = GridSearchCV(estimator=NNetRregCV, param_grid=param_grid, cv=10,n_jobs=-1)
CV_nnmodel.fit(X_train_nn, Y_train_nn)

This code performs automated hyperparameter tuning using 'GridSearchCV' for a neural network regression model trained on one-hot encoded features.
1. Line1: Wraps the 'MLPRegressor' in a 'GridSearchCV' object to perform an exhaustive search over a grid of hyperparameters
2. Line2: Trains the model using the training data

#### Calculating r2 score for Neural Network

In [ ]:
Y_train_pred = CV_nnmodel.predict(X_train_nn)
Y_train_dev = sum((Y_train_nn - Y_train_pred)**2)
Y_train_meandev = sum((Y_train_nn - Y_train_nn.mean())**2)  # [aus PDF]
r2 = 1 - Y_train_dev / Y_train_meandev  # [aus PDF]

This block manually computes the r2 score for the neural network model trained on one-hot encoded features using the training dataset
1. Line1: Predicts the target values on the training data using the best neural network selected by 'GridSearchCV'
2. Line2: Calculates the residual sum of squares (RSS) which represents the total prediction error of the model on the training set.
3. Line3: Calculates the total sum of squares (TSS) which represents the total variance in the training data assuming a baseline model that always predicts the mean
4. Line4: Computes the r2 score which measures how well the model explains the variance in the target variable

#### Calculating pseudo r2 score for Neural Network

In [ ]:
Y_test_pred = CV_nnmodel.predict(X_test_nn)
Y_test_dev = sum((Y_test_nn - Y_test_pred)**2)
Y_train_meandev = sum((Y_test_nn - Y_test_nn.mean())**2)  # [aus PDF]
pseudor2 = 1 - Y_test_dev / Y_train_meandev

This code block calculates the r2 score on the test dataset (pseudo r2) to evaluate the generalization performance of the neural network trained on one-hot encoded features.
1. Line1: Uses the trained neural network to predict values for the test set.
2. Line2: Computes the residual sum of squares (RSS) on the test set and measures the total prediction error made by the model on unseen data
3. Line3: Calculates the total sum of squares (TSS) on the test data
4. Line4: Calculates the test r2 (pseudo r2) which reflects how well the model generalizes to new data

#### Logging Neural Network Results to the Report

In [ ]:
DataPrep.report.loc[len(DataPrep.report)] = ["NN_OH ", r2, pseudor2,"", CV_nnmodel.cv_results_['mean_test_score'][CV_nnmodel.best_index_], CV_nnmodel.cv_results_['std_test_score'][CV_nnmodel.best_index_]]
print(DataPrep.report.head())
print(CV_nnmodel.best_params_)

This block adds a new row to your 'DataPrep.report' table to document the performance of a neural network model trained on one-hot encoded features and then displays the first few rows of the report and the best hyperparameters found during grid search.